In [ ]:
# =========================
# ✅ SINGLE-ACTIVITY LOSO ONLY (CSV 저장 X)  — A안 (Baseline은 windowing 사용 X)
#
# - Activity loop
# - For each activity and each fold, train ONE model only on that activity from 9 subjects (windows)
# - Test: evaluate that activity on held-out subject
#
# Integrator-defense modes:
#   orig / reverse / win_shuffle / block_shuffle / phase_randomize
#
# ✅ Baseline-A (Constant-rate integrator)  [ACT-ONLY + TRIAL-BASED]  ✅ windowing 사용 X
# - per-activity mean rate estimated from TRAIN TRIALS (NOT windows)
#   mean_rate = mean_i( count_i / duration_i ) over train trials
# - test count = mean_rate(act) * duration_test
# - ignores input/time order => (given same T) identical across modes
#
# =========================

import os
import glob
import random
import zlib
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Stable RNG helpers
# ---------------------------------------------------------------------
def _stable_seed_from_text(text: str, base_seed: int = 0) -> int:
    h = zlib.adler32(text.encode("utf-8"))
    return int((base_seed + h) % (2**31 - 1))


# ---------------------------------------------------------------------
# 3) Transformations (integrator-defense modes)
# ---------------------------------------------------------------------
def transform_reverse(x_np: np.ndarray) -> np.ndarray:
    return x_np[::-1].copy()


def transform_block_shuffle(x_np: np.ndarray, block_len: int, rng: np.random.RandomState) -> np.ndarray:
    """
    Split time into blocks, shuffle block order (keeps within-block local structure).
    x_np: (T,C)
    """
    x = np.asarray(x_np, dtype=np.float32)
    T, C = x.shape
    if block_len <= 1 or T <= block_len:
        return x.copy()

    n_blocks = int(np.ceil(T / block_len))
    blocks = []
    for bi in range(n_blocks):
        st = bi * block_len
        ed = min((bi + 1) * block_len, T)
        blocks.append(x[st:ed])

    perm = rng.permutation(len(blocks))
    out = np.concatenate([blocks[i] for i in perm], axis=0)

    if out.shape[0] != T:
        out = out[:T]
    return out.astype(np.float32, copy=False)


def transform_phase_randomize(x_np: np.ndarray, rng: np.random.RandomState) -> np.ndarray:
    """
    Phase randomization per channel:
    - preserve FFT magnitude
    - randomize phase (except DC & Nyquist), reconstruct real signal
    """
    x = np.asarray(x_np, dtype=np.float32)
    T, C = x.shape
    out = np.zeros_like(x)

    for c in range(C):
        sig = x[:, c].astype(np.float64)

        X = np.fft.rfft(sig)
        mag = np.abs(X)
        phase = np.angle(X)
        n = len(X)

        if n <= 2:
            rec = np.fft.irfft(X, n=T)
            out[:, c] = rec.astype(np.float32)
            continue

        rand_phase = rng.uniform(-np.pi, np.pi, size=n)
        rand_phase[0] = phase[0]
        rand_phase[-1] = phase[-1]

        newX = mag * (np.cos(rand_phase) + 1j * np.sin(rand_phase))
        rec = np.fft.irfft(newX, n=T)
        out[:, c] = rec.astype(np.float32)

    return out


# ---------------------------------------------------------------------
# 4) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    """
    label_config: list of (subj, act_id, gt_count)
    returns list of dict:
      data: (T,C) normalized per trial
      count: float
      subj, act_id, act_name, meta
    """
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_id = int(act_id)
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # trial-wise z-score
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                "data": norm_np,
                "count": float(gt_count),
                "subj": subj,
                "act_id": act_id,
                "act_name": act_name,
                "meta": f"{subj}_{act_name}",
            })
        else:
            print(f"[Skip] Missing data for {subj} - act_id={act_id} ({act_name})")

    return trial_list


# ---------------------------------------------------------------------
# 5) Windowing (TRAIN)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN: trial -> windows
    window label = trial avg rate * window duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "subj": item["subj"],
                "act_id": item["act_id"],
                "act_name": item["act_name"],
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "subj": item["subj"],
                "act_id": item["act_id"],
                "act_name": item["act_name"],
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "subj": item["subj"],
                    "act_id": item["act_id"],
                    "act_name": item["act_name"],
                })

    return windows


# ---------------------------------------------------------------------
# 6) Windowing inference (TEST) + win_shuffle
# ---------------------------------------------------------------------
def predict_count_by_windowing(
    model, x_np, fs, win_sec, stride_sec, device,
    tau=1.0, batch_size=64,
    win_shuffle=False, rng: np.random.RandomState = None
):
    """
    TEST: trial -> windows inference -> mean rate -> total count
    win_shuffle=True: shuffle time indices inside each window (strong order destruction).
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N,W,C)

    if win_shuffle:
        assert rng is not None
        N, W, C = windows.shape
        for i in range(N):
            perm = rng.permutation(W)
            windows[i] = windows[i, perm, :]

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N,C,W)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 7) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C,T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),
        "mask": torch.stack(masks),
        "count": torch.stack(counts),
        "length": torch.tensor(lengths, dtype=torch.float32),
        "meta": metas
    }


# ---------------------------------------------------------------------
# 8) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B,D,T)
        z = z.transpose(1, 2)      # (B,T,D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B,D,T)
        x_hat = self.net(zt)       # (B,C,T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp bias

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, _ = self.rate_head(z, tau=tau)
        micro_rate_t = amp_t             # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,)

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "phase_p": phase_p,          # (B,T,K)
            "rep_rate_t": rep_rate_t,    # (B,T)
            "k_hat": k_hat,              # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 9) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)
    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean()


# ---------------------------------------------------------------------
# 10) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.0075)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()
        rate_hat, _, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()


# ---------------------------------------------------------------------
# 11) Metrics helpers (entropy / effK)
# ---------------------------------------------------------------------
def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def compute_effK_from_phase_mean(phase_p_np, eps=1e-6):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    p_bar = phase_p_np.mean(axis=0)
    effK = 1.0 / (np.sum(p_bar ** 2) + eps)
    return float(effK)


# ---------------------------------------------------------------------
# 12) ✅ Baseline-A: TRIAL-BASED mean rate (windowing 사용 X)
# ---------------------------------------------------------------------
def estimate_baselineA_mean_rate_from_train_trials(train_trials, fs, eps=1e-6):
    """
    train_trials: list of dict (from prepare_trial_list)
      - each has: data (T,C), count (float)
    return: mean of TRIAL-level rates over ALL train trials
      rate_i = count_i / (T_i/fs)
    """
    rates = []
    for tr in train_trials:
        T = int(tr["data"].shape[0])
        dur = max(T / float(fs), eps)
        rates.append(float(tr["count"]) / dur)
    return float(np.mean(rates)) if len(rates) > 0 else 0.0


def baselineA_predict_count(mean_rate, T, fs):
    dur = max(T / float(fs), 1e-6)
    return float(mean_rate * dur)


# ---------------------------------------------------------------------
# 13) Core evaluator for ONE trial
# ---------------------------------------------------------------------
def eval_one_trial_all_modes(model, CONFIG, device, item, mean_rate_baseA, fold_idx, track_name="single"):
    MODES = ["orig", "reverse", "win_shuffle", "block_shuffle", "phase_randomize"]

    x_np_orig = item["data"]
    T = x_np_orig.shape[0]
    gt = float(item["count"])
    act_id = int(item["act_id"])
    act_name = item["act_name"]
    meta = item["meta"]
    test_subj = item["subj"]

    base_seed = int(CONFIG["seed"] + (fold_idx + 1) * 100000)
    rng_wshf  = np.random.RandomState(_stable_seed_from_text(meta + "__win_shuffle", base_seed))
    rng_bshf  = np.random.RandomState(_stable_seed_from_text(meta + "__block_shuffle", base_seed))
    rng_ph    = np.random.RandomState(_stable_seed_from_text(meta + "__phase_rand", base_seed))

    # ✅ Baseline-A: mode-agnostic (TRIAL-based mean rate)
    mean_rate_used = float(mean_rate_baseA)
    pred_base = baselineA_predict_count(mean_rate_used, T=T, fs=CONFIG["fs"])

    rows = []
    for mode in MODES:
        if mode == "orig":
            x_for_pred = x_np_orig
            win_shuffle = False
            rng = None
            x_for_aux = x_np_orig

        elif mode == "reverse":
            x_for_pred = transform_reverse(x_np_orig)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred

        elif mode == "win_shuffle":
            x_for_pred = x_np_orig
            win_shuffle = True
            rng = rng_wshf
            perm = rng_wshf.permutation(x_np_orig.shape[0])
            x_for_aux = x_np_orig[perm].copy()

        elif mode == "block_shuffle":
            block_len = int(round(CONFIG["block_sec"] * CONFIG["fs"]))
            x_for_pred = transform_block_shuffle(x_np_orig, block_len=block_len, rng=rng_bshf)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred

        elif mode == "phase_randomize":
            x_for_pred = transform_phase_randomize(x_np_orig, rng=rng_ph)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred

        else:
            raise ValueError(mode)

        pred_ours, _ = predict_count_by_windowing(
            model,
            x_np=x_for_pred,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            device=device,
            tau=CONFIG.get("tau", 1.0),
            batch_size=CONFIG.get("batch_size", 64),
            win_shuffle=win_shuffle,
            rng=rng
        )

        # aux (k_hat / entropy / effK) — full-trial 1회 forward
        x_tensor = torch.tensor(x_for_aux, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
        with torch.no_grad():
            _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))
        k_hat = float(aux["k_hat"].item())
        phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
        ent = compute_phase_entropy_mean(phase_p)
        effK = compute_effK_from_phase_mean(phase_p)

        dur = T / float(CONFIG["fs"])
        pred_rate = pred_ours / max(dur, 1e-6)
        gt_rate = gt / max(dur, 1e-6)

        abs_err = abs(pred_ours - gt)
        mape = abs_err / (abs(gt) + 1e-6) * 100.0
        bias = pred_ours - gt

        abs_err_b = abs(pred_base - gt)
        mape_b = abs_err_b / (abs(gt) + 1e-6) * 100.0
        bias_b = pred_base - gt

        rows.append({
            "track": track_name,
            "fold": fold_idx + 1,
            "test_subj": test_subj,
            "act_id": act_id,
            "act_name": act_name,
            "mode": mode,
            "T": int(T),
            "duration_sec": float(dur),
            "GT": float(gt),

            "Pred_ours": float(pred_ours),
            "MAE_ours": float(abs_err),
            "MAPE_ours": float(mape),
            "Bias_ours": float(bias),
            "GT_rate": float(gt_rate),
            "Pred_rate": float(pred_rate),
            "k_hat": float(k_hat),
            "entropy": float(ent),
            "effK": float(effK),

            "Pred_baseA": float(pred_base),
            "MAE_baseA": float(abs_err_b),
            "MAPE_baseA": float(mape_b),
            "Bias_baseA": float(bias_b),

            "BaseA_rate_trial": float(mean_rate_used),
        })

    return rows


# ---------------------------------------------------------------------
# 14) SINGLE-ACTIVITY LOSO (activity loop) ONLY
# ---------------------------------------------------------------------
def run_single_activity_loso_only(CONFIG, full_data, device, activity_specs):
    subjects = [f"subject{i}" for i in range(1, 11)]
    out_rows = []

    print("\n" + "=" * 140)
    print(" >>> SINGLE-ACTIVITY LOSO ONLY")
    print(" >>> Baseline-A: ACT-only + TRIAL-based mean rate (windowing 사용 X)")
    print("=" * 140)

    for spec in activity_specs:
        act_id = int(spec["act_id"])
        act_name = spec["act_name"]
        labels = spec["labels"]

        print("\n" + "-" * 140)
        print(f"[Single] Activity: {act_name} (id={act_id})")
        print("-" * 140)

        for fold_idx, test_subj in enumerate(subjects):
            set_strict_seed(CONFIG["seed"])

            train_labels = [t for t in labels if t[0] != test_subj]
            test_labels  = [t for t in labels if t[0] == test_subj]

            # Restrict maps to that activity only
            target_map_one = {act_id: act_name}
            feat_map_one   = {act_id: CONFIG["ACT_FEATURE_MAP"][act_id]}

            train_trials = prepare_trial_list(train_labels, full_data, target_map_one, feat_map_one)
            test_trials  = prepare_trial_list(test_labels,  full_data, target_map_one, feat_map_one)

            if len(train_trials) == 0 or len(test_trials) == 0:
                print(f"[Skip] Fold {fold_idx+1}: train_trials={len(train_trials)}, test_trials={len(test_trials)}")
                continue

            # ✅ OURS Train windows (contribution 유지)
            train_windows = trial_list_to_windows(
                train_trials, fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"], stride_sec=CONFIG["stride_sec"],
                drop_last=CONFIG["drop_last"]
            )

            # ✅ Baseline-A: TRIAL-based mean rate (windowing 사용 X)
            mean_rate_baseA = estimate_baselineA_mean_rate_from_train_trials(train_trials, fs=CONFIG["fs"])

            g = torch.Generator()
            g.manual_seed(CONFIG["seed"])

            train_loader = DataLoader(
                TrialDataset(train_windows),
                batch_size=CONFIG["batch_size"],
                shuffle=True,
                collate_fn=collate_variable_length,
                generator=g,
                num_workers=0
            )

            input_ch = train_windows[0]["data"].shape[1]
            model = KAutoCountModel(
                input_ch=input_ch,
                hidden_dim=CONFIG["hidden_dim"],
                latent_dim=CONFIG["latent_dim"],
                K_max=CONFIG["K_max"]
            ).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

            for epoch in range(CONFIG["epochs"]):
                train_one_epoch(model, train_loader, optimizer, CONFIG, device)
                scheduler.step()

            model.eval()

            fold_rows = []
            for item in test_trials:
                fold_rows.extend(eval_one_trial_all_modes(
                    model=model, CONFIG=CONFIG, device=device,
                    item=item, mean_rate_baseA=mean_rate_baseA,
                    fold_idx=fold_idx, track_name="single"
                ))
            out_rows.extend(fold_rows)

            # Fold quick summary
            df_fold = pd.DataFrame(fold_rows)
            gsum = (df_fold.groupby("mode")
                    .agg(n=("MAE_ours", "count"),
                         MAE_ours=("MAE_ours", "mean"),
                         MAE_baseA=("MAE_baseA", "mean"),
                         baseA_rate_trial=("BaseA_rate_trial", "mean"))
                    .reset_index())
            mae0 = float(gsum[gsum["mode"] == "orig"]["MAE_ours"].values[0]) if (gsum["mode"] == "orig").any() else float("nan")

            base_rate = float(df_fold["BaseA_rate_trial"].iloc[0])
            print(f"[Fold {fold_idx+1:2d}] Test={test_subj} | BaseA mean_rate(TRIAL)={base_rate:.4f} reps/s")
            for _, rr in gsum.iterrows():
                d = float(rr["MAE_ours"]) - mae0 if np.isfinite(mae0) else float("nan")
                print(f"  {rr['mode']:14s} | MAE ours={rr['MAE_ours']:.3f} (Δ={d:+.3f}) | MAE baseA={rr['MAE_baseA']:.3f} | n={int(rr['n'])}")

    return pd.DataFrame(out_rows)


# ---------------------------------------------------------------------
# 15) Print-only summaries (NO CSV saving)
# ---------------------------------------------------------------------
def print_summaries(df_all: pd.DataFrame):
    if df_all is None or len(df_all) == 0:
        print("[Warn] No rows to summarize.")
        return

    # Per-activity, per-mode
    g_act = (df_all.groupby(["act_id", "act_name", "mode"])
             .agg(
                n=("MAE_ours", "count"),
                MAE_ours=("MAE_ours", "mean"),
                MAE_baseA=("MAE_baseA", "mean"),
                entropy=("entropy", "mean"),
                effK=("effK", "mean"),
                k_hat=("k_hat", "mean"),
                baseA_rate_trial=("BaseA_rate_trial", "mean"),
             )
             .reset_index())

    # Δ vs orig inside each activity
    blocks = []
    for (aid, aname), sdf in g_act.groupby(["act_id", "act_name"], sort=False):
        base = sdf[sdf["mode"] == "orig"]
        mae0 = float(base["MAE_ours"].values[0]) if len(base) > 0 else np.nan
        tmp = sdf.copy()
        tmp["dMAE_vs_orig"] = tmp["MAE_ours"] - mae0
        blocks.append(tmp)
    g_act2 = pd.concat(blocks, axis=0).reset_index(drop=True)

    # Overall by mode
    g_all = (df_all.groupby(["mode"])
             .agg(
                n=("MAE_ours", "count"),
                MAE_ours=("MAE_ours", "mean"),
                MAE_baseA=("MAE_baseA", "mean"),
                entropy=("entropy", "mean"),
                effK=("effK", "mean"),
                k_hat=("k_hat", "mean"),
                baseA_rate_trial=("BaseA_rate_trial", "mean"),
             )
             .reset_index())
    if (g_all["mode"] == "orig").any():
        mae0 = float(g_all[g_all["mode"] == "orig"]["MAE_ours"].values[0])
        g_all["dMAE_vs_orig"] = g_all["MAE_ours"] - mae0
    else:
        g_all["dMAE_vs_orig"] = np.nan

    print("\n" + "=" * 140)
    print("[OVERALL SUMMARY]  (Δ = MAE(mode) - MAE(orig))")
    with pd.option_context('display.max_rows', 200, 'display.max_columns', 80, 'display.width', 220):
        print(g_all.sort_values("mode").to_string(index=False))

    print("\n" + "=" * 140)
    print("[PER-ACTIVITY SUMMARY]  (Δ = MAE(mode) - MAE(orig within activity))")
    with pd.option_context('display.max_rows', 999, 'display.max_columns', 80, 'display.width', 240):
        print(g_act2.sort_values(["act_id", "mode"]).to_string(index=False))
    print("=" * 140)


# ---------------------------------------------------------------------
# 16) Main
# ---------------------------------------------------------------------
def main():
    BASE_CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # training
        "epochs": 50,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # windowing (OURS에만 사용)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # loss weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        "tau": 1.0,

        # defense config
        "block_sec": 0.8,
    }

    DEFAULT_FEATS = [
        'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
        'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
        'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
        'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
        'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
    ]

    ACTIVITY_SPECS = [
        {"act_id": 6,  "act_name": "Waist bends forward",       "labels": [
            ("subject1", 6, 21), ("subject2", 6, 19), ("subject3", 6, 21), ("subject4", 6, 20), ("subject5", 6, 20),
            ("subject6", 6, 20), ("subject7", 6, 20), ("subject8", 6, 21), ("subject9", 6, 21), ("subject10", 6, 20),
        ]},
        {"act_id": 7,  "act_name": "Frontal elevation of arms", "labels": [
            ("subject1", 7, 20), ("subject2", 7, 20), ("subject3", 7, 20), ("subject4", 7, 20), ("subject5", 7, 20),
            ("subject6", 7, 20), ("subject7", 7, 20), ("subject8", 7, 19), ("subject9", 7, 19), ("subject10", 7, 20),
        ]},
        {"act_id": 8,  "act_name": "Knees bending",             "labels": [
            ("subject1", 8, 20), ("subject2", 8, 21), ("subject3", 8, 21), ("subject4", 8, 19), ("subject5", 8, 20),
            ("subject6", 8, 20), ("subject7", 8, 21), ("subject8", 8, 21), ("subject9", 8, 21), ("subject10", 8, 21),
        ]},
        {"act_id": 12, "act_name": "Jump front & back",         "labels": [
            ("subject1", 12, 20), ("subject2", 12, 22), ("subject3", 12, 21), ("subject4", 12, 21), ("subject5", 12, 20),
            ("subject6", 12, 21), ("subject7", 12, 19), ("subject8", 12, 20), ("subject9", 12, 20), ("subject10", 12, 20),
        ]},
        {"act_id": 10, "act_name": "Jogging",                   "labels": [
            ("subject1", 10, 157), ("subject2", 10, 161), ("subject3", 10, 154), ("subject4", 10, 154), ("subject5", 10, 160),
            ("subject6", 10, 156), ("subject7", 10, 153), ("subject8", 10, 160), ("subject9", 10, 166), ("subject10", 10, 156),
        ]},
        {"act_id": 11, "act_name": "Running",                   "labels": [
            ("subject1", 11, 165), ("subject2", 11, 158), ("subject3", 11, 174), ("subject4", 11, 163), ("subject5", 11, 157),
            ("subject6", 11, 172), ("subject7", 11, 149), ("subject8", 11, 166), ("subject9", 11, 174), ("subject10", 11, 172),
        ]},
    ]

    # Maps (load 단계에서 필요)
    target_map = {int(s["act_id"]): s["act_name"] for s in ACTIVITY_SPECS}
    feat_map = {int(s["act_id"]): DEFAULT_FEATS for s in ACTIVITY_SPECS}

    CONFIG = dict(BASE_CONFIG)
    CONFIG["TARGET_ACTIVITIES_MAP"] = target_map
    CONFIG["ACT_FEATURE_MAP"] = feat_map

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    # Load full data once
    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    # Run SINGLE only
    df_all = run_single_activity_loso_only(CONFIG, full_data, device, ACTIVITY_SPECS)

    # Print summaries (NO CSV saving)
    print_summaries(df_all)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> SINGLE-ACTIVITY LOSO ONLY
 >>> Baseline-A: ACT-only + TRIAL-based mean rate (windowing 사용 X)

--------------------------------------------------------------------------------------------------------------------------------------------
[Single] Activity: Waist bends forward (id=6)
--------------------------------------------------------------------------------------------------------------------------------------------
[Fold  1] Test=subject1 | BaseA mean_rate(TRIAL)=0.3698 reps/s
  block_shuffle  | MAE ours=1.820 (Δ=-0.318) | MAE baseA=1.718 | n=1
  orig           | MAE ours=2.139 (Δ=+0.000) | MAE baseA=1.718 | n=1
  phase_randomize | MAE ours=6.618 (Δ=+4.480) | MAE baseA=1.718 | n=1
  reverse        | MAE ours=1.730 (Δ=-0.408) | MAE baseA=1.718 | n=1
  win_shuffle    | MAE ours=3.910 (Δ=+1.771) | MAE baseA=1.718 | n=1
[Fold  2] Test=subject2 | BaseA mean_rate(TRIAL)=0.3745 rep

In [3]:
# =========================
# ✅ SINGLE-ACTIVITY LOSO ONLY (CSV 저장 X) — OURS + Baseline-E2 only
#
# - Activity loop
# - For each activity and each fold:
#   - Train OURS on that activity from 9 subjects (windows)
#   - Test on held-out subject (full trial)
#
# Integrator-defense modes:
#   orig / reverse / win_shuffle / block_shuffle / phase_randomize
#
# ✅ Baseline-E2 (ACT-ONLY + TRIAL-BASED; "textbook integrator" flavor)
#   - Full-trial energy integral on RAW (non-zscored) after DC removal:
#       E = (1/fs) * sum_t sum_c x_dc(t,c)^2
#   - Linear calibration with beta=0:
#       count ≈ alpha * E
#   - Uses RAW signal (transformed per mode for consistency)
# =========================

import os
import glob
import random
import zlib
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Stable RNG helpers
# ---------------------------------------------------------------------
def _stable_seed_from_text(text: str, base_seed: int = 0) -> int:
    h = zlib.adler32(text.encode("utf-8"))
    return int((base_seed + h) % (2**31 - 1))


# ---------------------------------------------------------------------
# 3) Transformations (integrator-defense modes)
# ---------------------------------------------------------------------
def transform_reverse(x_np: np.ndarray) -> np.ndarray:
    return x_np[::-1].copy()


def transform_block_shuffle(x_np: np.ndarray, block_len: int, rng: np.random.RandomState) -> np.ndarray:
    """
    Split time into blocks, shuffle block order (keeps within-block local structure).
    x_np: (T,C)
    """
    x = np.asarray(x_np, dtype=np.float32)
    T, C = x.shape
    if block_len <= 1 or T <= block_len:
        return x.copy()

    n_blocks = int(np.ceil(T / block_len))
    blocks = []
    for bi in range(n_blocks):
        st = bi * block_len
        ed = min((bi + 1) * block_len, T)
        blocks.append(x[st:ed])

    perm = rng.permutation(len(blocks))
    out = np.concatenate([blocks[i] for i in perm], axis=0)

    if out.shape[0] != T:
        out = out[:T]
    return out.astype(np.float32, copy=False)


def transform_phase_randomize(x_np: np.ndarray, rng: np.random.RandomState) -> np.ndarray:
    """
    Phase randomization per channel:
    - preserve FFT magnitude
    - randomize phase (except DC & Nyquist), reconstruct real signal
    """
    x = np.asarray(x_np, dtype=np.float32)
    T, C = x.shape
    out = np.zeros_like(x)

    for c in range(C):
        sig = x[:, c].astype(np.float64)

        X = np.fft.rfft(sig)
        mag = np.abs(X)
        phase = np.angle(X)
        n = len(X)

        if n <= 2:
            rec = np.fft.irfft(X, n=T)
            out[:, c] = rec.astype(np.float32)
            continue

        rand_phase = rng.uniform(-np.pi, np.pi, size=n)
        rand_phase[0] = phase[0]
        rand_phase[-1] = phase[-1]

        newX = mag * (np.cos(rand_phase) + 1j * np.sin(rand_phase))
        rec = np.fft.irfft(newX, n=T)
        out[:, c] = rec.astype(np.float32)

    return out


# ---------------------------------------------------------------------
# 4) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    """
    label_config: list of (subj, act_id, gt_count)
    returns list of dict:
      data: (T,C) normalized per trial  (for model)
      raw : (T,C) raw float32           (for baseline integrator)
      count: float
      subj, act_id, act_name, meta
    """
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_id = int(act_id)
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # trial-wise z-score (MODEL INPUT)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                "data": norm_np,
                "raw": raw_np,
                "count": float(gt_count),
                "subj": subj,
                "act_id": act_id,
                "act_name": act_name,
                "meta": f"{subj}_{act_name}",
            })
        else:
            print(f"[Skip] Missing data for {subj} - act_id={act_id} ({act_name})")

    return trial_list


# ---------------------------------------------------------------------
# 5) Windowing (TRAIN) — for OURS model
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN: trial -> windows
    window label = trial avg rate * window duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "subj": item["subj"],
                "act_id": item["act_id"],
                "act_name": item["act_name"],
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "subj": item["subj"],
                "act_id": item["act_id"],
                "act_name": item["act_name"],
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "subj": item["subj"],
                    "act_id": item["act_id"],
                    "act_name": item["act_name"],
                })

    return windows


# ---------------------------------------------------------------------
# 6) Windowing inference (TEST) + win_shuffle (OURS)
# ---------------------------------------------------------------------
def predict_count_by_windowing(
    model, x_np, fs, win_sec, stride_sec, device,
    tau=1.0, batch_size=64,
    win_shuffle=False, rng: np.random.RandomState = None
):
    """
    TEST: trial -> windows inference -> mean rate -> total count
    win_shuffle=True: shuffle time indices inside each window.
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N,W,C)

    if win_shuffle:
        assert rng is not None
        N, W, C = windows.shape
        for i in range(N):
            perm = rng.permutation(W)
            windows[i] = windows[i, perm, :]

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N,C,W)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 7) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C,T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),
        "mask": torch.stack(masks),
        "count": torch.stack(counts),
        "length": torch.tensor(lengths, dtype=torch.float32),
        "meta": metas
    }


# ---------------------------------------------------------------------
# 8) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B,D,T)
        z = z.transpose(1, 2)      # (B,T,D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B,D,T)
        x_hat = self.net(zt)       # (B,C,T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp bias

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, _ = self.rate_head(z, tau=tau)
        micro_rate_t = amp_t             # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,)

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "phase_p": phase_p,          # (B,T,K)
            "rep_rate_t": rep_rate_t,    # (B,T)
            "k_hat": k_hat,              # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 9) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)
    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean()


# ---------------------------------------------------------------------
# 10) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.0075)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()
        rate_hat, _, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()


# ---------------------------------------------------------------------
# 11) Metrics helpers (entropy / effK)
# ---------------------------------------------------------------------
def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def compute_effK_from_phase_mean(phase_p_np, eps=1e-6):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    p_bar = phase_p_np.mean(axis=0)
    effK = 1.0 / (np.sum(p_bar ** 2) + eps)
    return float(effK)


# ---------------------------------------------------------------------
# 12) ✅ Baseline-E2: FULL-TRIAL energy integrator (RAW) + calibration (beta=0)
# ---------------------------------------------------------------------
def compute_energy_integral_raw(x_raw: np.ndarray, fs: int, eps=1e-12) -> float:
    """
    x_raw: (T,C) raw float32
    - DC removal (per-channel mean subtraction)
    - energy integral: (1/fs) * sum_t sum_c (x_dc^2)
    """
    x = np.asarray(x_raw, dtype=np.float32)
    x = x - x.mean(axis=0, keepdims=True)
    E = float(np.sum(x * x) / max(float(fs), eps))
    return E


def fit_linear_energy_calibrator_alpha_only(train_trials, fs: int, ridge: float = 1e-8):
    """
    Fit alpha with beta=0: y ≈ alpha*E
    alpha = (E^T y) / (E^T E + ridge)
    """
    if len(train_trials) == 0:
        return 0.0

    Es, ys = [], []
    for tr in train_trials:
        E = compute_energy_integral_raw(tr["raw"], fs=fs)
        Es.append(E)
        ys.append(float(tr["count"]))

    Es = np.asarray(Es, dtype=np.float64)
    ys = np.asarray(ys, dtype=np.float64)

    num = float(np.sum(Es * ys))
    den = float(np.sum(Es * Es) + ridge)
    return float(num / den)


def baselineE2_predict_count(alpha_only: float, x_raw: np.ndarray, fs: int) -> float:
    E = compute_energy_integral_raw(x_raw, fs=fs)
    return float(alpha_only * E)


# ---------------------------------------------------------------------
# 13) Core evaluator for ONE trial (OURS + E2)
# ---------------------------------------------------------------------
def eval_one_trial_all_modes(
    model, CONFIG, device, item,
    baseE2_alpha,
    fold_idx, track_name="single"
):
    MODES = ["orig", "reverse", "win_shuffle", "block_shuffle", "phase_randomize"]

    x_np_orig = item["data"]   # normalized (for model)
    x_raw_orig = item["raw"]   # raw (for baseline)

    T = x_np_orig.shape[0]
    gt = float(item["count"])
    act_id = int(item["act_id"])
    act_name = item["act_name"]
    meta = item["meta"]
    test_subj = item["subj"]

    base_seed = int(CONFIG["seed"] + (fold_idx + 1) * 100000)
    rng_wshf  = np.random.RandomState(_stable_seed_from_text(meta + "__win_shuffle", base_seed))
    rng_bshf  = np.random.RandomState(_stable_seed_from_text(meta + "__block_shuffle", base_seed))
    rng_ph    = np.random.RandomState(_stable_seed_from_text(meta + "__phase_rand", base_seed))

    rows = []
    for mode in MODES:
        # -------------------------
        # model input transform (normalized) + baseline raw transform
        # -------------------------
        if mode == "orig":
            x_for_pred = x_np_orig
            win_shuffle = False
            rng = None
            x_for_aux = x_np_orig
            x_raw_for_base = x_raw_orig

        elif mode == "reverse":
            x_for_pred = transform_reverse(x_np_orig)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred
            x_raw_for_base = transform_reverse(x_raw_orig)

        elif mode == "win_shuffle":
            x_for_pred = x_np_orig
            win_shuffle = True
            rng = rng_wshf

            perm_full = rng_wshf.permutation(x_np_orig.shape[0])
            x_for_aux = x_np_orig[perm_full].copy()
            x_raw_for_base = x_raw_orig[perm_full].copy()  # (E2는 순서불변이라 값은 사실 동일)

        elif mode == "block_shuffle":
            block_len = int(round(CONFIG["block_sec"] * CONFIG["fs"]))
            x_for_pred = transform_block_shuffle(x_np_orig, block_len=block_len, rng=rng_bshf)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred
            x_raw_for_base = transform_block_shuffle(x_raw_orig, block_len=block_len, rng=rng_bshf)

        elif mode == "phase_randomize":
            x_for_pred = transform_phase_randomize(x_np_orig, rng=rng_ph)
            win_shuffle = False
            rng = None
            x_for_aux = x_for_pred
            x_raw_for_base = transform_phase_randomize(x_raw_orig, rng=rng_ph)

        else:
            raise ValueError(mode)

        # -------------------------
        # OURS prediction
        # -------------------------
        pred_ours, _ = predict_count_by_windowing(
            model,
            x_np=x_for_pred,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            device=device,
            tau=CONFIG.get("tau", 1.0),
            batch_size=CONFIG.get("batch_size", 64),
            win_shuffle=win_shuffle,
            rng=rng
        )

        # aux (k_hat / entropy / effK) — full-trial 1회 forward
        x_tensor = torch.tensor(x_for_aux, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
        with torch.no_grad():
            _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))
        k_hat = float(aux["k_hat"].item())
        phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
        ent = compute_phase_entropy_mean(phase_p)
        effK = compute_effK_from_phase_mean(phase_p)

        dur = T / float(CONFIG["fs"])
        pred_rate = pred_ours / max(dur, 1e-6)
        gt_rate = gt / max(dur, 1e-6)

        abs_err = abs(pred_ours - gt)
        mape = abs_err / (abs(gt) + 1e-6) * 100.0
        bias = pred_ours - gt

        # -------------------------
        # Baseline-E2 prediction
        # -------------------------
        pred_baseE2 = baselineE2_predict_count(baseE2_alpha, x_raw_for_base, fs=int(CONFIG["fs"]))
        abs_err_b = abs(pred_baseE2 - gt)
        mape_b = abs_err_b / (abs(gt) + 1e-6) * 100.0
        bias_b = pred_baseE2 - gt

        rows.append({
            "track": track_name,
            "fold": fold_idx + 1,
            "test_subj": test_subj,
            "act_id": act_id,
            "act_name": act_name,
            "mode": mode,
            "T": int(T),
            "duration_sec": float(dur),
            "GT": float(gt),

            "Pred_ours": float(pred_ours),
            "MAE_ours": float(abs_err),
            "MAPE_ours": float(mape),
            "Bias_ours": float(bias),
            "GT_rate": float(gt_rate),
            "Pred_rate": float(pred_rate),
            "k_hat": float(k_hat),
            "entropy": float(ent),
            "effK": float(effK),

            "Pred_baseE2": float(pred_baseE2),
            "MAE_baseE2": float(abs_err_b),
            "MAPE_baseE2": float(mape_b),
            "Bias_baseE2": float(bias_b),
            "baseE2_alpha": float(baseE2_alpha),
        })

    return rows


# ---------------------------------------------------------------------
# 14) SINGLE-ACTIVITY LOSO (activity loop) ONLY
# ---------------------------------------------------------------------
def run_single_activity_loso_only(CONFIG, full_data, device, activity_specs):
    subjects = [f"subject{i}" for i in range(1, 11)]
    out_rows = []

    print("\n" + "=" * 140)
    print(" >>> SINGLE-ACTIVITY LOSO ONLY")
    print(" >>> Baseline: E2(full-trial energy integrator, beta=0) | OURS(window-stabilized rate learning)")
    print("=" * 140)

    for spec in activity_specs:
        act_id = int(spec["act_id"])
        act_name = spec["act_name"]
        labels = spec["labels"]

        print("\n" + "-" * 140)
        print(f"[Single] Activity: {act_name} (id={act_id})")
        print("-" * 140)

        for fold_idx, test_subj in enumerate(subjects):
            set_strict_seed(CONFIG["seed"])

            train_labels = [t for t in labels if t[0] != test_subj]
            test_labels  = [t for t in labels if t[0] == test_subj]

            # Restrict maps to that activity only
            target_map_one = {act_id: act_name}
            feat_map_one   = {act_id: CONFIG["ACT_FEATURE_MAP"][act_id]}

            train_trials = prepare_trial_list(train_labels, full_data, target_map_one, feat_map_one)
            test_trials  = prepare_trial_list(test_labels,  full_data, target_map_one, feat_map_one)

            if len(train_trials) == 0 or len(test_trials) == 0:
                print(f"[Skip] Fold {fold_idx+1}: train_trials={len(train_trials)}, test_trials={len(test_trials)}")
                continue

            # ✅ OURS Train windows
            train_windows = trial_list_to_windows(
                train_trials, fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"], stride_sec=CONFIG["stride_sec"],
                drop_last=CONFIG["drop_last"]
            )

            # ✅ Baseline-E2: fit on TRAIN TRIALS only
            baseE2_alpha = fit_linear_energy_calibrator_alpha_only(train_trials, fs=int(CONFIG["fs"]))

            g = torch.Generator()
            g.manual_seed(CONFIG["seed"])

            train_loader = DataLoader(
                TrialDataset(train_windows),
                batch_size=CONFIG["batch_size"],
                shuffle=True,
                collate_fn=collate_variable_length,
                generator=g,
                num_workers=0
            )

            input_ch = train_windows[0]["data"].shape[1]
            model = KAutoCountModel(
                input_ch=input_ch,
                hidden_dim=CONFIG["hidden_dim"],
                latent_dim=CONFIG["latent_dim"],
                K_max=CONFIG["K_max"]
            ).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

            for epoch in range(CONFIG["epochs"]):
                train_one_epoch(model, train_loader, optimizer, CONFIG, device)
                scheduler.step()

            model.eval()

            fold_rows = []
            for item in test_trials:
                fold_rows.extend(eval_one_trial_all_modes(
                    model=model, CONFIG=CONFIG, device=device,
                    item=item,
                    baseE2_alpha=baseE2_alpha,
                    fold_idx=fold_idx, track_name="single"
                ))
            out_rows.extend(fold_rows)

            # Fold quick summary
            df_fold = pd.DataFrame(fold_rows)
            gsum = (df_fold.groupby("mode")
                    .agg(
                        n=("MAE_ours", "count"),
                        MAE_ours=("MAE_ours", "mean"),
                        MAE_baseE2=("MAE_baseE2", "mean"),
                    )
                    .reset_index())

            mae0 = float(gsum[gsum["mode"] == "orig"]["MAE_ours"].values[0]) if (gsum["mode"] == "orig").any() else float("nan")
            print(f"[Fold {fold_idx+1:2d}] Test={test_subj} | E2(alpha={baseE2_alpha:.3g}, beta=0)")
            for _, rr in gsum.iterrows():
                d = float(rr["MAE_ours"]) - mae0 if np.isfinite(mae0) else float("nan")
                print(f"  {rr['mode']:14s} | MAE ours={rr['MAE_ours']:.3f} (Δ={d:+.3f}) | MAE E2={rr['MAE_baseE2']:.3f} | n={int(rr['n'])}")

    return pd.DataFrame(out_rows)


# ---------------------------------------------------------------------
# 15) Print-only summaries (NO CSV saving)
# ---------------------------------------------------------------------
def print_summaries(df_all: pd.DataFrame):
    if df_all is None or len(df_all) == 0:
        print("[Warn] No rows to summarize.")
        return

    # Overall by mode
    g_all = (df_all.groupby(["mode"])
             .agg(
                n=("MAE_ours", "count"),
                MAE_ours=("MAE_ours", "mean"),
                MAE_baseE2=("MAE_baseE2", "mean"),
                entropy=("entropy", "mean"),
                effK=("effK", "mean"),
                k_hat=("k_hat", "mean"),
                baseE2_alpha=("baseE2_alpha", "mean"),
             )
             .reset_index())

    if (g_all["mode"] == "orig").any():
        mae0 = float(g_all[g_all["mode"] == "orig"]["MAE_ours"].values[0])
        g_all["dMAE_vs_orig"] = g_all["MAE_ours"] - mae0
    else:
        g_all["dMAE_vs_orig"] = np.nan

    print("\n" + "=" * 140)
    print("[OVERALL SUMMARY]  (Δ = MAE(mode) - MAE(orig))")
    with pd.option_context('display.max_rows', 200, 'display.max_columns', 80, 'display.width', 220):
        print(g_all.sort_values("mode").to_string(index=False))

    # Per-activity, per-mode
    g_act = (df_all.groupby(["act_id", "act_name", "mode"])
             .agg(
                n=("MAE_ours", "count"),
                MAE_ours=("MAE_ours", "mean"),
                MAE_baseE2=("MAE_baseE2", "mean"),
                entropy=("entropy", "mean"),
                effK=("effK", "mean"),
                k_hat=("k_hat", "mean"),
             )
             .reset_index())

    blocks = []
    for (aid, aname), sdf in g_act.groupby(["act_id", "act_name"], sort=False):
        base = sdf[sdf["mode"] == "orig"]
        mae0 = float(base["MAE_ours"].values[0]) if len(base) > 0 else np.nan
        tmp = sdf.copy()
        tmp["dMAE_vs_orig"] = tmp["MAE_ours"] - mae0
        blocks.append(tmp)
    g_act2 = pd.concat(blocks, axis=0).reset_index(drop=True)

    print("\n" + "=" * 140)
    print("[PER-ACTIVITY SUMMARY]  (Δ = MAE(mode) - MAE(orig within activity))")
    with pd.option_context('display.max_rows', 999, 'display.max_columns', 80, 'display.width', 240):
        print(g_act2.sort_values(["act_id", "mode"]).to_string(index=False))
    print("=" * 140)


# ---------------------------------------------------------------------
# 16) Main
# ---------------------------------------------------------------------
def main():
    BASE_CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # training
        "epochs": 50,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # windowing (OURS)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # loss weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        "tau": 1.0,

        # defense config
        "block_sec": 0.8,
    }

    DEFAULT_FEATS = [
        'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
        'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
        'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
        'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
        'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
    ]

    ACTIVITY_SPECS = [
        {"act_id": 6,  "act_name": "Waist bends forward",       "labels": [
            ("subject1", 6, 21), ("subject2", 6, 19), ("subject3", 6, 21), ("subject4", 6, 20), ("subject5", 6, 20),
            ("subject6", 6, 20), ("subject7", 6, 20), ("subject8", 6, 21), ("subject9", 6, 21), ("subject10", 6, 20),
        ]},
        {"act_id": 7,  "act_name": "Frontal elevation of arms", "labels": [
            ("subject1", 7, 20), ("subject2", 7, 20), ("subject3", 7, 20), ("subject4", 7, 20), ("subject5", 7, 20),
            ("subject6", 7, 20), ("subject7", 7, 20), ("subject8", 7, 19), ("subject9", 7, 19), ("subject10", 7, 20),
        ]},
        {"act_id": 8,  "act_name": "Knees bending",             "labels": [
            ("subject1", 8, 20), ("subject2", 8, 21), ("subject3", 8, 21), ("subject4", 8, 19), ("subject5", 8, 20),
            ("subject6", 8, 20), ("subject7", 8, 21), ("subject8", 8, 21), ("subject9", 8, 21), ("subject10", 8, 21),
        ]},
        {"act_id": 12, "act_name": "Jump front & back",         "labels": [
            ("subject1", 12, 20), ("subject2", 12, 22), ("subject3", 12, 21), ("subject4", 12, 21), ("subject5", 12, 20),
            ("subject6", 12, 21), ("subject7", 12, 19), ("subject8", 12, 20), ("subject9", 12, 20), ("subject10", 12, 20),
        ]},
        {"act_id": 10, "act_name": "Jogging",                   "labels": [
            ("subject1", 10, 157), ("subject2", 10, 161), ("subject3", 10, 154), ("subject4", 10, 154), ("subject5", 10, 160),
            ("subject6", 10, 156), ("subject7", 10, 153), ("subject8", 10, 160), ("subject9", 10, 166), ("subject10", 10, 156),
        ]},
        {"act_id": 11, "act_name": "Running",                   "labels": [
            ("subject1", 11, 165), ("subject2", 11, 158), ("subject3", 11, 174), ("subject4", 11, 163), ("subject5", 11, 157),
            ("subject6", 11, 172), ("subject7", 11, 149), ("subject8", 11, 166), ("subject9", 11, 174), ("subject10", 11, 172),
        ]},
    ]

    target_map = {int(s["act_id"]): s["act_name"] for s in ACTIVITY_SPECS}
    feat_map = {int(s["act_id"]): DEFAULT_FEATS for s in ACTIVITY_SPECS}

    CONFIG = dict(BASE_CONFIG)
    CONFIG["TARGET_ACTIVITIES_MAP"] = target_map
    CONFIG["ACT_FEATURE_MAP"] = feat_map

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    df_all = run_single_activity_loso_only(CONFIG, full_data, device, ACTIVITY_SPECS)
    print_summaries(df_all)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> SINGLE-ACTIVITY LOSO ONLY
 >>> Baseline: E2(full-trial energy integrator, beta=0) | OURS(window-stabilized rate learning)

--------------------------------------------------------------------------------------------------------------------------------------------
[Single] Activity: Waist bends forward (id=6)
--------------------------------------------------------------------------------------------------------------------------------------------
[Fold  1] Test=subject1 | E2(alpha=0.00643, beta=0)
  block_shuffle  | MAE ours=1.820 (Δ=-0.318) | MAE E2=3.376 | n=1
  orig           | MAE ours=2.139 (Δ=+0.000) | MAE E2=3.376 | n=1
  phase_randomize | MAE ours=6.618 (Δ=+4.480) | MAE E2=3.376 | n=1
  reverse        | MAE ours=1.730 (Δ=-0.408) | MAE E2=3.376 | n=1
  win_shuffle    | MAE ours=3.910 (Δ=+1.771) | MAE E2=3.376 | n=1
[Fold  2] Test=subject2 | E2(alpha=0.00625, beta=0)
  bl